# Эксперименты с MST

In [2]:
import numpy as np
import pandas as pd
from datetime import datetime
import networkx as nx
import matplotlib.pyplot as plt
import k_scripts.finance_scripts as fs
import k_scripts.network_scripts as ns
import k_scripts.sampler_scripts as ss
import k_scripts.measures_scripts as ms
import k_scripts.experiments_script as es
import seaborn as sn
from tqdm import tqdm
import importlib
import json 

importlib.reload(ns)
importlib.reload(es)
importlib.reload(ms)

<module 'k_scripts.measures_scripts' from '/home/friday/_homescript/measures-of-uncertainty/k_scripts/measures_scripts.py'>

In [62]:
path = './data/NASDAQ/NASDAQ_18_19_Close.csv'
dClose = pd.read_csv(path,index_col='Date')
R = fs.get_daily_returns(dClose)

In [66]:
assert not dClose.isnull().values.any()

In [67]:
R.head(5)

,AAL,AAPL,AMD,AMZN,ARCC,ATHX,ATRA,ATVI,AVGO,AXAS,...,SQQQ,TOPS,TQQQ,TRNX,VISL,VXRT,WORX,XOG,XSPA,ZN
Date,,,,,,,,,,,,,,,,,,,,,
2018-01-03,-0.012342,-0.000174,0.050610,0.012694,-0.004407,0.005510,0.061599,0.015430,0.010876,-0.003914,...,-0.029838,0.083382,0.028858,-0.030772,0.000000,0.018349,0.098940,0.012237,-0.006920,0.017168
2018-01-04,0.006285,0.004634,0.048172,0.004466,-0.003160,0.016349,-0.067139,-0.010002,0.000333,0.007813,...,-0.005061,0.076961,0.005845,0.089612,0.006231,0.018018,0.056353,0.006734,-0.035339,0.016878
2018-01-05,-0.000380,0.011321,-0.020001,0.016033,-0.000633,0.031918,0.027399,0.026102,0.005908,-0.043745,...,-0.030391,-0.076961,0.029754,-0.058841,0.006192,0.000000,0.027029,-0.012154,0.089407,-0.047118
2018-01-08,-0.009926,-0.003721,0.033116,0.014322,0.003162,0.060933,0.002699,0.003910,0.002390,0.020121,...,-0.011573,0.039221,0.011060,-0.030772,0.006154,0.017700,-0.047791,-0.030347,-0.068053,0.017392
2018-01-09,-0.000960,-0.000115,-0.038179,0.004665,0.001892,-0.040206,0.042223,-0.006625,-0.013943,0.000000,...,-0.000529,-0.039221,0.000763,0.000000,-0.037504,0.017392,-0.520193,-0.026252,-0.096015,0.017094


In [68]:
R.shape

(470, 236)

In [69]:
Corr = R.corr()
Cov = R.cov()

## Нормальное распределение

In [71]:
exp_path = './experiments/NASDAQ/mst/Normal'

In [72]:
sampler = np.random.multivariate_normal
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0] # СРЕДНЕЕ НОЛЬ


In [74]:
E_measures = dict()
N_min=10
N_max=20
step = 10

for N in tqdm(range(N_min,N_max,step)):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, N])
    
file =  exp_path + f'/mst_normal_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

100%|██████████| 1/1 [06:26<00:00, 386.89s/it]


## Распределение Стьюдента

In [8]:
sampler = ss.multivariate_t
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0]
df=3

In [10]:
E_measures = dict()
N_min=200
N_max=400
step = 100

for i,N in tqdm(enumerate(range(2,5,1))):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, df, N])
    print(i,N)
    
file =  f'./experiments/t_mst_DE/mst_t_{df}_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

1it [00:43, 43.34s/it]

0 2


2it [01:28, 43.85s/it]

1 3


3it [02:14, 44.90s/it]

2 4


## Смешанное распределение

In [6]:
sampler = ss.mixed_t_normal
reference_network = ns.create_network(R.corr().values)
mean = [0] * Cov.shape[0]
df=3
rat = 0.5

In [7]:
E_measures = dict()
N_min=200
N_max=400
step = 100

for N in tqdm(range(N_min,N_max,step)):
    E_measures[int(N)] = ms.E_measure_MST(reference_network, sampler, sampler_params=[mean, Cov, df, N,rat])
    
file =  f'./experiments/mixed_mst_DE/mst_mixed_{df}_{rat}_{N_min}-{N_max}.json'
with open(file, 'w') as f:
    json.dump(E_measures, f) 

100%|██████████| 2/2 [01:28<00:00, 44.50s/it]


# NASDAQ